### Tutorial: Model Comparison and Assessment for Microtubule Catastrophe

#### Written by : Joeyta Banerjee, Rashi Jeeda, and Mei Yi You

This notebook is a guide to using the code provided in our package. This example illustrates how we have used our package to analyze our microtubule catastrophe data. 

The first thing we would need to do is to clone the package from git and install using ```pip install -e MCAT_pkg```

In [38]:
%load_ext autoreload
%autoreload 2
import os, sys
# Using alias for packages can reduce having to write out the entire name every single time you call them
import bokeh
import pandas as pd
import holoviews as hv
from bokeh.layouts import row, column
import MCAT_pkg as mc 
from bokeh.plotting import save

import warnings
warnings.simplefilter('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


You can find out more information about a package or function by adding "?" or "??" behind its name:

In [2]:
mc?

Type:        module
String form: <module 'MCAT_pkg' from 'c:\\users\\rashi jeeda\\documents\\caltech\\classes\\3-1-bebi103a\\mcat\\mcat_pkg\\MCAT_pkg\\__init__.py'>
File:        c:\users\rashi jeeda\documents\caltech\classes\3-1-bebi103a\mcat\mcat_pkg\mcat_pkg\__init__.py
Docstring:  
The contents of this package allow users to reproduce our microtubule catastrophe analysis. It includes:
Our methods for parsing the raw data
Methods for performing exploratory analysis
Statistical analysis modules (bootstrapping, hypothesis testing, MLE analysis, and model assessment)


After we import our package, we will need to load in our data. 

In [3]:
mc??

Type:        module
String form: <module 'MCAT_pkg' from 'c:\\users\\rashi jeeda\\documents\\caltech\\classes\\3-1-bebi103a\\mcat\\mcat_pkg\\MCAT_pkg\\__init__.py'>
File:        c:\users\rashi jeeda\documents\caltech\classes\3-1-bebi103a\mcat\mcat_pkg\mcat_pkg\__init__.py
Source:     
"""The contents of this package allow users to reproduce our microtubule catastrophe analysis. It includes:
Our methods for parsing the raw data
Methods for performing exploratory analysis
Statistical analysis modules (bootstrapping, hypothesis testing, MLE analysis, and model assessment)
"""
from .parsing import *
from .exploratory_analysis import *
from .bootstrapping import *
from .MLE_analysis import *
from .model_assessment import *
__author__ = 'Joeyta Banerjee'
__email__ = 'jbanerje@caltech.edu'
__version__ = '0.0.1'


In [4]:
# Identify the location that the data file is in  
data_path = "data/gardner_mt_catastrophe_only_tubulin.csv"

# Read in the file as df
df = pd.read_csv(data_path, comment = "#")

Let's take a look at the first 5 rows of the data using the command:

In [5]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35.0,25.0,50.0,60.0
1,40.000,45.0,40.0,60.0,75.0
2,40.000,50.0,40.0,60.0,75.0
3,45.429,50.0,45.0,75.0,85.0
4,50.000,55.0,50.0,75.0,115.0


We would like to parse the dataframe into a tidy format. For more information about tidy data, go to the link [here.](https://bebi103a.github.io/lessons/06/tidy_data.html)

In [6]:
# Use mc.parse_df to parse the data into working format from the file path
df_parsed = mc.parse_df("data/gardner_mt_catastrophe_only_tubulin.csv")

Let's take a look at the dataframe.

In [7]:
df_parsed.head()

,Concentration (uM),Time to catastrophe (s)
0,12 uM,25.000
1,12 uM,40.000
2,12 uM,40.000
3,12 uM,45.429
4,12 uM,50.000


Next, we would like to define a categorical color palette to use for different tubulin concentrations

In [8]:
# Define a color palette to use throughout plots, one color per experiment trial
colors = bokeh.palettes.YlGn[6]
print(colors)

('#006837', '#31a354', '#78c679', '#addd8e', '#d9f0a3', '#ffffcc')


We would now perform exploratory analysis on the dataset using an ECDF, since they visualize the cumulative density function (CDF). For more information, visit the link [here.](https://bebi103a.github.io/lessons/06/visualizing_distributions.html)

In [9]:
p = mc.categorical_plot(df_parsed, variable = "Time to catastrophe (s)", cats = "Concentration (uM)", 
                  palette = colors, order = ['14 uM', '12 uM', '10 uM', '9 uM', '7 uM'])
p.title.text = "ECDF of Catastrophe times by Concentration"
mc.bokeh.io.show(p)

From the ECDF, we can see that the tubulin concentration 14uM has the longest overall time to catastrophe, followed by 12 uM, 10uM, and 7uM/9uM. We can see a general trend that higher tubulin concentrations tend to have longer catastrophe times. 

We now want to see what the individual tubulin concentrations would look like as stripbox plots. 

In [10]:
p_strip = mc.categorical_plot(df_parsed, variable = "Time to catastrophe (s)", cats = "Concentration (uM)", 
                              format = "stripbox",
                             order=['14 uM', '12 uM', '10 uM', '9 uM', '7 uM'],
                            palette = colors)
p_strip.title.text = "Stripbox Plot of Catastrophe Times, separated by Concentration"
mc.bokeh.io.show(p_strip)

Here, we can see a similar trend that higher tubulin concentrations tend to have longer catastrophe times. One thing to note, however, is that catastrophe times of tubulin concentration 12uM has more outliers, which can influence/bias the mean. 

Now, we would like to compare whether a Gamma distribution or an exponential distribution is a better model for this data. For simplicity, we will first perform the tests on tubulin concentration 12uM and then the rest of the concentrations once we determine which model is better. 

We need to pull out the data for catastrophe times for a tubulin concentration of 12 uM.

In [11]:
# Generate array of times to catastrophe for 12 uM concentration of tubulin
conc_12_times = df_parsed.loc[df_parsed["Concentration (uM)"] == '12 uM', 'Time to catastrophe (s)'].values

#### Gamma Distribution Model
We will first generate parameter estimates according to the Gamma distribution. You can look at the functions within the MCAT_pgk folder. 

In [12]:
mc.mle_iid_gamma(conc_12_times)

array([2.91493987, 0.00766123])

Perform maximum likelihood estimates for parameters for i.i.d. gamma measurements, parametrized by alpha, b=1/beta

In [13]:
mc.log_like_iid_gamma_log_params(mc.mle_iid_gamma(conc_12_times), conc_12_times)

-4637.174686632476

We will draw bootstrap replicates of our MLE. 

In [14]:
gamma_bs_reps = mc.draw_bs_reps(conc_12_times, mc.mle_iid_gamma, size=100)

This will give us the MLE estimate.

In [15]:
gamma_mle = mc.np.mean(gamma_bs_reps, axis = 0)

In [34]:
p = mc.overlay_models(conc_12_times,  "Time to catastrophe (s)", gamma_mle, 
                      exp_color = "#addd8e",
                     theor_color = "dimgray")
p.title.text = "Gamma Theoretical Distribution"
bokeh.io.show(p)

#### Exponential Distribution Model

We will move on to generate parameter estimates according to the exponential distribution.

To obtain MLEs for both beta values, we begin with the formula for the PDF : $f(t; \beta_1, \beta_2) = \frac{\beta_1 \beta_2}{\beta_2 - \beta_1}(e^{-\beta_1t} - e^{-\beta_2t})$. We want to find the MLE, so we will consider the problem of maximizing the log likelihood rather than attempting to maximize this expression and avoid errors with dividing by zero when $\beta_1 = \beta_2$. However, we will first rewrite this expression by setting a new variable, $\Delta \beta = \beta_2 - \beta_1$ to simplify some of the algebraic manipulation. We also do this to avoid the potential of underflowing errors in the code when we get extremely small values of $\beta$. This gives us: 

$L(t; \beta_1, \Delta \beta) = \prod_{i = 1}^n\frac{(\beta_1)(\beta_1 + \Delta\beta)}{\Delta\beta}(e^{-\beta_1 t})(1 - e^{-\Delta \beta})$

Taking the natural logarithm of this expression for likelihood, the product will turn into a summation, which is much easier to work with. We get:

$l(t; \beta_1, \Delta \beta) = \sum_{i = 1}^n \mbox{ln}\beta_1 +\mbox{ln}(\beta_1 + \Delta \beta) - \mbox{ln}\Delta \beta - \beta t_i + \mbox{ln}(1 - e^{-\Delta \beta t_i})$

This is the form for log likelihood that we will use below.

In [17]:
# For example sake, we will only be bootstrapping 100 replicates 
# instead of 10000 that we used in our analysis
exp_bs_reps = mc.draw_bs_reps(conc_12_times, mc.mle_iid_exp, size=100)

In [18]:
exp_mle = mc.np.mean(exp_bs_reps, axis = 0)

In [19]:
exp_mle

array([5.27416552e-03, 6.25540129e-06])

In [36]:
p1 = mc.overlay_models(conc_12_times,  "Time to catastrophe (s)", exp_mle, 
                      cdf_fun = mc.model_cdf,
                      exp_color = "#addd8e",
                     theor_color = "dimgray")
p1.title.text = "Exponential Theoretical Distribution"
bokeh.io.show(p1)

In [37]:
c = row(p, p1)
bokeh.io.show(c)

In [39]:
save(c, 'C:/Users/Rashi Jeeda/Documents/Caltech/Classes/3-1-BEBi103a/rjeeda.github.io/_includes/MLE_comparison.html')

'C:\\Users\\Rashi Jeeda\\Documents\\Caltech\\Classes\\3-1-BEBi103a\\rjeeda.github.io\\_includes\\MLE_comparison.html'

We want to get the AIC values for each distribution 

In [21]:
# Exponential distribution
mc.AIC(exp_mle, mc.log_like_iid_exp_log_params, conc_12_times)

9327.393774606271

In [22]:
# Gamma distribution 
mc.AIC(gamma_mle, mc.MLE_analysis.log_like_iid_gamma_log_params, conc_12_times)

9278.3572422719

We can now plot the QQ plot for the exponential model. 

In [23]:
p = mc.QQ_plot(conc_12_times, mc.gen_exponential, exp_mle, size = 10000, 
               axis_label = "Time to catastrophe (s)", title = "Exponential QQ-Plot", color = "#78c679")
p.background_fill_color = '#fafafa'
mc.bokeh.io.show(p)

The QQ plot for the gamma model.

In [24]:
p2 = mc.QQ_plot(conc_12_times, mc.gen_gamma, gamma_mle, size = 10000, 
                axis_label = "Time to catastrophe (s)", title = "Gamma QQ-Plot",
               color = "#78c679")
p2.background_fill_color = '#fafafa'
mc.bokeh.io.show(p2)

We would also like to compare the ECDF of theoretical distribution to experimental for each model. 

In [25]:
# Exponential distribution 
p3 = mc.predictive_ecdf(conc_12_times, mc.gen_exponential, exp_mle, title = 'Exponential Distribution ECDF', 
                        color = "green")
p3.background_fill_color = '#fafafa'
mc.bokeh.io.show(p3)

In [26]:
# Gamma distribution 
p4 = mc.predictive_ecdf(conc_12_times, mc.gen_gamma, gamma_mle, title = 'Gamma Distribution ECDF', 
                        color = "green")
p4.background_fill_color = '#fafafa'
mc.bokeh.io.show(p4)

Since the AIC of the exponential distribution (9327.391952721562) is higher than that of the gamma distribution (9278.358353623804), the gamma model is preferred. Based on the plots for predictive ECDF, we see that the gamma distribution is more closely aligned with the experimental results. Similarly, from the QQ-plots, we can see that there is more separation of the generative quantiles from the observed quantiles in the exponential distribution model than the gamma distribution model. As a result, based on AIC (Akaike information criterion) and QQ plot, the gamma distribution model is better model. 

### Parameter Estimates for the Other Tubulin Concentrations  

Now that we have a preferred model, we will obtain parameter estimates for the other tubulin concentrations using the exponential model. 

In [27]:
# Get unique concentrations and take out 12uM
all_concentrations = df_parsed["Concentration (uM)"].unique()
concentrations = mc.np.delete(all_concentrations, 0)
all_other_bs_reps = []
all_other_conf_int = []
all_other_mean = []

In [28]:
# Generate bootstrap replicates for the other tubulin concentrations using the exponential model
# Generate array of times to catastrophe for other concentrations of tubulin
for conc in concentrations:
    conc_times = df_parsed.loc[df_parsed["Concentration (uM)"] == conc, 
                                  'Time to catastrophe (s)'].values
    bs_reps = mc.draw_bs_reps(conc_times, mc.mle_iid_gamma, size=100)
    all_other_bs_reps.append(bs_reps)
    mean = mc.np.mean(bs_reps, axis = 0)
    all_other_mean.append(mean)
    conf_int = mc.np.percentile(bs_reps, [2.5, 97.5], axis=0)
    all_other_conf_int.append(conf_int)
    print('The MLE for number of arrivals for a catastrophe to occur (b_1) and difference in b_1 and b_2 (delta_b) is respectively {} with a 95% confidence interval of \n{}\n'
          .format(mean, conf_int))

The MLE for number of arrivals for a catastrophe to occur (b_1) and difference in b_1 and b_2 (delta_b) is respectively [2.46636274 0.0076392 ] with a 95% confidence interval of 
[[2.23975738 0.00677078]
 [2.68335442 0.00848151]]

The MLE for number of arrivals for a catastrophe to occur (b_1) and difference in b_1 and b_2 (delta_b) is respectively [2.69446756 0.00881559] with a 95% confidence interval of 
[[2.3888131  0.00784617]
 [3.01236238 0.01013395]]



KeyboardInterrupt: 

From the plot of $\beta_1$ and $\Delta\beta$ of the different tubulin concentrations, we can see that higher tubulin concentrations seem to have lower $\beta_1$ values (1/(time to first catastrophe)). All the mean $\beta_1$values for the different tubulin concentrations, however, are all on the order of 1e-3. In general, by looking at the values of the parameters, we see that the $\Delta\beta$ values are on the order of 0 to 1e-4 or 1e-5, which means that $\beta_1$ and $\beta_2$ are often the same or are very similar in value, indicating that catastrophe times one after another are often very similar and may be dependent. 

In [ ]:
%load_ext watermark
%watermark -v -p MCAT_pkg,numpy,scipy,pandas,bokeh,iqplot,tqdm,jupyterlab

Acknowledgements!

We thank the publishers of Gardner et. al. for sharing their data, the BeBi103 TA’s for their guidance, the makers of Poole for this website template, Rosita Fu and Griffin Chure for design inspiration, and of course, Justin Bois for his assistance, insight, and useful code!